In [23]:
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream
import pandas as pd
import sys

reload(sys)
sys.setdefaultencoding("utf-8")

ckey = 'ALCoPyhzrbOTIKrFsFjFEA'
csecret = 'XMbqXddwhpyA50ac9bctZmNgek62BkD9zmrUvEMbc'
atoken = '889624724-vesOkw7n4LI0npescKYNjiT2VDpcv1jP3MhzW45g'
asecret = 'k97A1nicLkekwKsUQaqgB0f9qvuHw0ln0ycsKq2JRi60E'

oauth = OAuth(atoken, asecret, ckey, csecret)

# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)

# Get a sample of the public data following through Twitter
iterator = twitter_stream.statuses.filter(locations='-126,-58,26,50', lang='en')

print "\nHow many tweets would you like to collect?"
tweet_count = 1000 # input()
output_file = 'testweets' # raw_input('Enter an output csv filename: ') or 'tweets.csv'

with open(str(output_file),'w') as tweet_file:
        #final_dict = {'uid':[], 'tid':[], 'text':[], 'timestamp':[], 'city':[], 'country':[], 'bounding_box':[]}
        final_dict = {'uid':[], 'tid':[], 'text':[], 'timestamp':[], 'city':[], 'state':[],'country':[], 'bounding_box':[]}
        for tweet in iterator:
                tweet_count -= 1
                # Twitter Python Tool wraps the data returned by Twitter
                # as a TwitterDictResponse object.
                # We convert it back to the JSON format to print/score
                tweet.values()
                for k,v in tweet.iteritems():
                        if k == 'text':
                                # ensuring tweet text is quoted for proper csv file output
                                #quoted_text =  str(v)
                                quoted_text =  '"' + v + '"' 
                                final_dict['text'].append(quoted_text)
                        elif k == 'user':
                                final_dict['uid'].append(v['id'])
                        elif k == 'id':
                                final_dict['tid'].append(v)
                        elif k == 'timestamp_ms':
                                final_dict['timestamp'].append(long(v))
                        elif k == 'place':
                                try:
                                        final_dict['city'].append(v['full_name'].split(',')[0])
                                        final_dict['state'].append(v['full_name'].split(',')[-1])
                                        final_dict['country'].append(v['country'])
                                        final_dict['bounding_box'].append(v['bounding_box'])
                                except:
                                        final_dict['city'].append('')
                                        final_dict['state'].append('')
                                        final_dict['country'].append('')
                                        final_dict['bounding_box'].append('')


                if tweet_count <= 0:
                        break


        tweet_df = pd.DataFrame(final_dict)
        tweet_df.to_csv(output_file + ".csv")
